In [0]:
#uncomment in case of google colab

# from google.colab import drive
# drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#uncomment in case of google colab

# !pip install filterpy

     |████████████████████████████████| 184kB 6.4MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-cp36-none-any.whl size=110453 sha256=4db3f894321f72bb920ad8029b4925f8d6f87eccac56c2c9880ba8e9a81451d4
  Stored in directory: /root/.cache/pip/wheels/c3/0c/dd/e92392c3f38a41371602d99fc77d6c1d42aadbf0c6afccdd02
Successfully built filterpy


In [0]:
#uncomment in case of google colab

# import os
# PATH_TO_FILE = '/content/drive/My Drive/Development/traffic_analysis'
# os.chdir(PATH_TO_FILE)

In [0]:
import colorsys
import os
from timeit import default_timer as timer
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.layers import Input, Conv2D, Add, ZeroPadding2D, UpSampling2D, Concatenate, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.regularizers import l2
import cv2
import os
from keras.utils import multi_gpu_model
from functools import reduce, wraps
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
from sort.sort import Sort

import warnings
warnings.filterwarnings('ignore')
np.set_printoptions(precision=3)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
#@title yolo

class YOLO(object):
    _defaults = {
        "model_path": 'model_data/yolo.h5',
        "anchors_path": 'model_data/yolo_anchors.txt',
        "classes_path": 'model_data/coco_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
        "text_size" : 2
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()

        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def predict(self, image):
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(np.copy(image), tuple(reversed(self.model_image_size)))
            image_data = boxed_image

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.shape[0], image.shape[1]],#[image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        
        return out_boxes, out_scores, out_classes

    def draw_boxes(self, image, out_boxes, out_scores, out_classes, indexes_to_highlite, ids, class_count, line_y, thicknessFactor = 4):
        image_h = image.shape[0]
        image_w = image.shape[1]
        white_color = (255, 255, 255)
        thickness = 1
        fontScale=1
        
        total = int(sum(class_count.values()))
        items_cnt = ['{}: {}'.format(self.class_names[key], int(val)) for key, val in class_count.items()]
        items_cnt_str = ', '.join(items_cnt)
        whole_count_text = f'total: {total}, {items_cnt_str}'
        cv2.putText(image, whole_count_text, (50, image_h - 50), cv2.FONT_HERSHEY_SIMPLEX, 
                    2*fontScale, white_color, int(thicknessFactor*thickness))
       
        #for i, obj_id in reversed(list(enumerate(ids))):
        for i, obj_id in enumerate(ids):
            predicted_class = self.class_names[out_classes[i]]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.shape[0], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.shape[1], np.floor(right + 0.5).astype('int32'))

            mid_h = (bottom-top)/2+top
            mid_v = (right-left)/2+left
            white_color = (255,255,255)
            black_color = (0,0,0)
            
            if mid_h >= line_y and i in indexes_to_highlite:
                cv2.rectangle(image, (left, top), (right, bottom), self.colors[out_classes[i]], int(thicknessFactor*thickness))
                # put text above rectangle
                cv2.putText(image, label, (left, top-2), cv2.FONT_HERSHEY_SIMPLEX, fontScale, white_color, 2)
            else:
                # put object rectangle
                cv2.rectangle(image, (left, top), (right, bottom), white_color, thickness)   

        return image

    def close_session(self):
        self.sess.close()

    def get_class_names(self):
        return self.class_names

In [0]:
#@title MOT
DEFAULT_MAX_DISTANCE_BETWEEN_POINTS = 60 
DEFAULT_WARMUP_FRAMES = 8
DEFAUTL_MIN_HITS = 3
DEFAULT_MAX_AGE = 10
class MOT():

    def __init__(self, **kwargs):
        print(kwargs)
        self._state = {}
        self._statistics = []
        self._class_count = dict(zip(kwargs['class_ids'], np.zeros(len(kwargs['class_ids']))))
        # if distance between centers of two bboxes is less than _max_distance then object is staying
        self._max_distance = kwargs['max_distance'] if 'max_distance' in kwargs else DEFAULT_MAX_DISTANCE_BETWEEN_POINTS

        # after _warmup_frames we start to compare bbox's centers for one tracked object
        self._warmup_frames = kwargs['warmup_frames'] if 'warmup_frames' in kwargs else DEFAULT_WARMUP_FRAMES

        self._line_y = kwargs['line_y'] if 'line_y' in kwargs else 0

        min_hits = kwargs['min_hits'] if 'min_hits' in kwargs else DEFAUTL_MIN_HITS
        max_age = kwargs['max_age'] if 'max_age' in kwargs else DEFAULT_MAX_AGE
        #self.display_config()
        self._mot_tracker = Sort(max_age, min_hits)

    def display_config(self):
        print('line_y')
        print(self._line_y)
        print('warmup_frames')
        print(self._warmup_frames)
        print('max_distance')
        print(self._max_distance)

    def update_state(self, boxes, scores, classes, timestamp):
        dets = np.array(boxes)
        dets = np.hstack((dets, scores.reshape(scores.shape[0],1)))
        trackers, matched, unmatched_dets = self._mot_tracker.update(dets)
        boxes, scores, classes, ids = self.mot_output_postprocess(trackers, boxes, scores, classes, matched, unmatched_dets)
        filtered_inds, object_crossed = self.filter_moving_obj_ids(boxes, scores, classes, ids)

        if len(object_crossed) > 0:
             self._statistics.append({'timestamp': timestamp, 'class_count': self._class_count.copy(), 'objects': object_crossed})
     
        scores = scores.reshape((scores.shape[0],))
        classes = classes.reshape((classes.shape[0],))
        classes = classes.astype(int)

        return filtered_inds, boxes, scores, classes, ids

    def filter_moving_obj_ids(self, boxes, scores, classes, ids):
        filtered_inds = set()
        object_crossed = []
        for i, obj_id in enumerate(ids):
            top, left, bottom, right = boxes[i]
            w = right - left
            h = bottom - top
            x_c = left + w/2
            y_c = top + h/2
            if obj_id in self._state:
                state_obj = self._state[obj_id]
                if state_obj['frame_num'] <  self._warmup_frames:
                    state_obj['frame_num']+=1
                    self._state[obj_id] = state_obj
                else:
                    if not self.is_close([x_c, y_c], state_obj['origin_pos']) and \
                        state_obj['origin_pos'][1] < y_c:
                        filtered_inds.add(i)
                
                        if not state_obj['already_counted']:
                            origin_y = state_obj['origin_pos'][1]
                        
                            if state_obj['origin_pos'][1] < self._line_y and y_c >= self._line_y:
                                self._class_count[classes[i]] += 1
                                state_obj['already_counted'] = True
                                self._state[obj_id] = state_obj
                                object_crossed.append([classes[i], scores[i]])
                    
            else:
                new_obj = {'frame_num': 1, 'origin_pos': [x_c, y_c], 'already_counted': False}
                self._state[obj_id] = new_obj

        return filtered_inds, object_crossed

    def mot_output_postprocess(self, trackers, boxes, scores, classes, matched, unmatched_dets):
        trackers =trackers[::-1]
      
        matched = matched[matched[:,1].argsort()]
        new_ind = matched[:,0]

        boxes_unmathced = np.empty((0,4))
        scores_unmathced = np.empty((0,1))
        classes_unmathced = np.empty((0,1))
        if len(unmatched_dets) > 0:
            boxes_unmathced = boxes.take(unmatched_dets, axis = 0)
            scores_unmathced = scores.take(unmatched_dets, axis = 0)
            classes_unmathced = classes.take(unmatched_dets, axis = 0)

        boxes = trackers[:, 0:4]
        scores = scores.take(new_ind, axis = 0)
        classes = classes.take(new_ind, axis = 0)
       
        ids = trackers[:, 4]

        scores = scores.reshape(-1,1)
        classes = classes.reshape(-1,1)
        scores_unmathced = scores_unmathced.reshape(-1,1)
        classes_unmathced = classes_unmathced.reshape(-1,1)

        boxes = np.vstack((boxes, boxes_unmathced))
        scores = np.vstack((scores, scores_unmathced))
        classes = np.vstack((classes, classes_unmathced))
        
        scores = scores.reshape((-1,))
        classes = classes.reshape((-1,))

        return boxes, scores, classes, ids
        
    def get_class_count(self):
        return self._class_count
    
    def get_statistics(self):
        return self._statistics

    def is_close(self, point_1, point_2):
        dist = np.linalg.norm(np.array(point_1)-np.array(point_2))
        return dist < self._max_distance
        

In [0]:
def draw_line(image, line_y, color, thickness):
    x1 = 0
    y1 = int(line_y)
    x2 = image.shape[1]
    y2 = int(line_y)
    cv2.line(image, (x1, y1), (x2, y2), color, thickness)
    return image

In [0]:
# config ={"model_path":'model_data/trained_weights_stage_1.h5',
#         "classes_path": 'model_data/vehicle_test_classes.txt'}
# yolo = YOLO(**config)
yolo = YOLO()

model_data/yolo.h5 model, anchors, and classes loaded.


In [0]:
# use to filter vehicle classes when use coco weights (80 classess)
def filter_vehicle_classes(boxes, scores, classes):
    vehicle_classes = {2, 5, 7}
    inds = np.arange(classes.shape[0])
    filtered_inds = [i for i in inds if classes[i] in vehicle_classes]
   
    boxes = boxes[filtered_inds]
    scores = scores[filtered_inds]
    classes = classes[filtered_inds]

    return boxes, scores, classes

In [0]:
def detect_video(yolo, input_video, output_video, h_reltive_pos, coco_weights=False):
    vid = cv2.VideoCapture(input_video)
    if not vid.isOpened():
        raise IOError("Couldn't open video")

    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    class_names = yolo.get_class_names()
    class_ids = np.arange(len(class_names))
    if coco_weights:
        class_names = ['car', 'bus', 'truck']
        class_ids = [2,5,7]

    line_y = video_size[1]*h_reltive_pos
    config = {'min_hits': 3, 'max_age':10, 'line_y': line_y,
              'max_distance': 90, 'warmup_frames': 10,
              'class_ids': class_ids}
    mot_tracker = MOT(**config)

    print("info: ", output_video, video_FourCC, video_fps, video_size)
    out = cv2.VideoWriter(output_video, cv2.VideoWriter_fourcc(*'MJPG'), video_fps, video_size)
   
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    nb_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'frames num = {nb_frames}')
    for i in range(nb_frames):
        return_value, frame = vid.read()
        boxes, scores, classes = yolo.predict(frame)
        
        if coco_weights:
            boxes, scores, classes = filter_vehicle_classes(boxes, scores, classes)

        filtered_inds, boxes, scores, classes, ids = mot_tracker.update_state(boxes, scores, classes, timestamp=int(i//video_fps))

        class_count = mot_tracker.get_class_count()
        image = yolo.draw_boxes(frame, boxes, scores, classes, filtered_inds, ids, class_count, line_y, 5)
        image = draw_line(image, line_y, color=(255,255,255), thickness=4)

        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        if i % 50 == 0:
            print(f'fps = {fps}')
        out.write(np.uint8(image))
    
    statistics = mot_tracker.get_statistics()

    vid.release()
    out.release()

    #write statistics to file
    statistics_path = str(output_video)[:-4] + '_statistics.txt'
    with open(statistics_path, 'w') as file:
        file.write(f'input video: {input_video}')
        file.write('\n')
        file.write(f'output video: {output_video}')
        file.write('\n')
        file.write(f'line position: {h_reltive_pos}')
        file.write('\n')
        for stat in statistics:
            timestamp = stat['timestamp']
            class_count = stat['class_count']
            total = int(sum(class_count.values()))
            if coco_weights:
                helper_dict = {2:0, 5:1, 7:2}
                items_cnt = ['{}: {}'.format(class_names[helper_dict[key]], int(val)) for key, val in class_count.items()]
                objects_str = '; '.join(list(map(lambda x: 'class: {0}; pred: {1}'.format(class_names[helper_dict[x[0]]], x[1]),
                                                 stat['objects'])))
            else:
                items_cnt = ['{}: {}'.format(class_names[key], int(val)) for key, val in class_count.items()]
                objects_str = '; '.join(list(map(lambda x: f'class: {int(x[0])}; pred: {x[1]}', stat['objects'])))
            items_cnt_str = ', '.join(items_cnt)
            whole_count_text = f'total: {total}, {items_cnt_str}'
            line = f'timestamp: {timestamp} sec.;    {whole_count_text};    objects: {objects_str}'
            file.write(line)
            file.write('\n')

            
    #yolo.close_session()

In [0]:
input_video = 'data/example2.mp4'
output_video = 'data/example2_proc.mp4'

In [0]:
h_reltive_pos = 0.33 # position of crossed line
detect_video(yolo, input_video, output_video, h_reltive_pos, coco_weights=True)